This file contains
1.simple model using ollama
2.RAG model using ollama
3.simple model using groq
4.RAG model using Groq

Setup wise, you likley to run into some errors. 
For ollama go to ollama.com downlaod the app and pull the models you want to use. This will take significant space so be weary.
For groq youll simply loop up groq api and make an account then put your api key in the api_key space, this runs faster but I did run out of alloated calls when doing some of my hw before. 


In [55]:
import os
!pip install langchain sentence-transformers transformers ollama
from langchain.llms import Ollama
import warnings
import time
import requests
import warnings
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from PyPDF2 import PdfReader

from groq import Groq

# Install necessary packages
!pip install langchain sentence-transformers transformers ollama

In [59]:
ollama_model = Ollama(model="llama3", temperature=0) 

def ask_question(question):
    response = ollama_model.invoke(question)
    return response

question = "What does Nutritional deficiencies and excesses arise from "
answer = ask_question(question)
print("Question:", question)
print("Answer:", answer)

Question: What does Nutritional deficiencies and excesses arise from 
Answer: Nutritional deficiencies and excesses can arise from a variety of factors, including:

**Deficiencies:**

1. **Inadequate diet**: A diet that lacks essential nutrients or is imbalanced can lead to deficiencies.
2. **Poor food absorption**: Conditions like celiac disease, Crohn's disease, or gastroparesis can impair nutrient absorption.
3. **Increased demand**: Certain situations, such as pregnancy, lactation, or illness, may increase the body's need for specific nutrients.
4. **Malabsorption syndromes**: Conditions like cystic fibrosis, short bowel syndrome, or pancreatic insufficiency can lead to malabsorption of nutrients.
5. **Gastrointestinal disorders**: Conditions like irritable bowel syndrome (IBS), inflammatory bowel disease (IBD), or gastrointestinal surgery can impair nutrient absorption.

**Excesses:**

1. **Overconsumption**: Consuming excessive amounts of certain nutrients, such as vitamins or mi

In [60]:
def load_pdf_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open("temp.pdf", "wb") as f:
            f.write(response.content)
        reader = PdfReader("temp.pdf")
        text = "".join([page.extract_text() for page in reader.pages])
        return text
    except requests.RequestException as e:
        print(f"Failed to download PDF from {url}: {str(e)}")
        return ""

def load_data():
    pdf_urls = [
        "https://www.biorxiv.org/content/10.1101/2020.07.28.224253v1.full.pdf",
        "https://www.cartercenter.org/resources/pdfs/health/ephti/library/lecture_notes/health_extension_trainees/generalpathology.pdf"
    ]

    documents_with_metadata = []
    for url in pdf_urls:
        data = load_pdf_from_url(url)
        if data:
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
            splits = text_splitter.split_text(data)

            for split in splits:
                documents_with_metadata.append(Document(page_content=split, metadata={"source": url}))

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    if not os.path.exists("chromadb"):
        os.makedirs("chromadb")

    vectorstore = Chroma.from_documents(
        documents=documents_with_metadata, 
        embedding=embedding_model, 
        persist_directory="chromadb"
    )

    return vectorstore

vectorstore = load_data()

ollama_model = Ollama(model="llama3", temperature=0)

def get_answer(query):
    template = """
    Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know; don't try to make up an answer.
    Keep the answer as concise as possible, preferably in three sentences.
    {context}
    Question: {question}
    Answer:
    """

    QA_CHAIN_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template,
    )

    retriever = vectorstore.as_retriever()
    context = retriever.invoke(query)
    context_text = "\n".join([doc.page_content for doc in context])

    prompt = QA_CHAIN_PROMPT.format(context=context_text, question=query)

    response = ollama_model.invoke(prompt)
    return response


question = "What does Nutritional deficiencies and excesses arise from "
answer = get_answer(question)
print("Question:", question)
print("Answer:", answer)

Question: What does Nutritional deficiencies and excesses arise from 
Answer: According to the provided context, nutritional deficiencies and excesses can arise from inefficient transport within the body or defective utilization. This may take the form of deficiency in major classes of food, such as protein and energy, or vitamins and elements essential for specific metabolic processes.


In [63]:
api_key = ""
#your api key


groq_client = Groq(api_key=api_key)

def query_llama3(question):
    response = groq_client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": question
            }
        ],
        model="llama3-8b-8192"
    )
    return response.choices[0].message.content

question = "What does Nutritional deficiencies and excesses arise from"
answer = query_llama3(question)
print("Question:", question)
print("Answer:", answer)

Question: Explain the importance of fast language models.
Answer: Fast language models, also known as efficient language models, are artificial intelligence (AI) models designed to process and generate human-like language quickly and with minimal computational resources. Their importance lies in their ability to enable various applications that rely on natural language processing (NLP) and machine learning (ML). Here are some key reasons why fast language models are crucial:

1. **Scalability and Speed**: Fast language models can handle large amounts of text data quickly, making them essential for tasks that require processing huge volumes of data, such as text classification, sentiment analysis, and language translation.
2. **Real-time Processing**: With the rise of real-time applications (e.g., chatbots, virtual assistants, and online search engines), fast language models enable instant text processing, providing a better user experience and enabling faster response times.
3. **Edge 

In [64]:
api_key = ""
#your api key

groq_client = Groq(api_key=api_key)

def load_pdf_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open("temp.pdf", "wb") as f:
            f.write(response.content)
        reader = PdfReader("temp.pdf")
        text = "".join([page.extract_text() for page in reader.pages])
        return text
    except requests.RequestException as e:
        print(f"Failed to download PDF from {url}: {str(e)}")
        return ""

def load_data():
    pdf_urls = [
        "https://www.biorxiv.org/content/10.1101/2020.07.28.224253v1.full.pdf",
        "https://www.cartercenter.org/resources/pdfs/health/ephti/library/lecture_notes/health_extension_trainees/generalpathology.pdf"
    ]

    documents_with_metadata = []
    for url in pdf_urls:
        data = load_pdf_from_url(url)
        if data:
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
            splits = text_splitter.split_text(data)

            for split in splits:
                documents_with_metadata.append(Document(page_content=split, metadata={"source": url}))

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    if not os.path.exists("chromadb"):
        os.makedirs("chromadb")

    vectorstore = Chroma.from_documents(
        documents=documents_with_metadata, 
        embedding=embedding_model, 
        persist_directory="chromadb"
    )

    return vectorstore

vectorstore = load_data()

def query_llama3_rag(question):
    template = """
    Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know; don't try to make up an answer.
    Keep the answer as concise as possible, preferably in three sentences.
    {context}
    Question: {question}
    Answer:
    """

    QA_CHAIN_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template,
    )

    retriever = vectorstore.as_retriever()
    context_docs = retriever.invoke(question)
    context_text = "\n".join([doc.page_content for doc in context_docs])

    prompt = QA_CHAIN_PROMPT.format(context=context_text, question=question)

    response = groq_client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="llama3-8b-8192"
    )
    return response.choices[0].message.content

question = "What does Nutritional deficiencies and excesses arise from"
answer = query_llama3_rag(question)
print("Question:", question)
print("Answer:", answer)

Question: What are the main findings of the first paper?
Answer: I'll answer your questions based on the given context.

1. What is the primary problem?
The primary problem is shock, specifically hypovolemic shock, given the patient's low blood pressure (60/20 mmHg).

2. What is the complication of the primary problem? Discuss the pathogenesis of this complication.
The complication of hypovolemic shock is organ failure, particularly renal failure. The pathogenesis is that the decreased blood flow and pressure lead to ischemia and potentially irreversible cellular damage in organs like the kidneys, causing acute kidney injury or failure.

3. What organisms are the most likely causes of the disease?
The organisms most likely causing the disease are likely to be gram-negative bacteria, such as Escherichia coli, Klebsiella pneumonia, or Pseudomonas aeruginosa, given the patient's altered consciousness, temperature of 38.9°C, and hypotension.

4. What are the morphologic changes and their c